In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch
import pickle


In [ ]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [ ]:
sats_games = []
PREFIXES = ['sep19_encumbered']
# PREFIXES = ['sep20']
for prefix in PREFIXES:
    sats_games += [f'{prefix}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{prefix}/*')]
    
sats_games = list(set(sats_games))

sats_games = [
        s for s in sats_games if ('undersell_allowed' in s) or ('tie_break' in s) or (s.endswith('base') or s.endswith('risk_averse') or s.endswith('spite')) or ('no_activity' in s) or ('grace_period' in s)
]

sats_games = [s for s in sats_games if 'undersell_allowed' in s]


print(len(sats_games))
print(sats_games)

75
['oct4_encumberedheuristic/oct4_encumberedheuristic_3_spite_grace_period', 'oct4_encumberedheuristic/oct4_encumberedheuristic_1_spite_undersell_allowed', 'oct4_encumberedheuristic/oct4_encumberedheuristic_4_base_tie_break', 'oct4_encumberedheuristic/oct4_encumberedheuristic_4_risk_averse_no_activity', 'oct4_encumberedheuristic/oct4_encumberedheuristic_1_risk_averse', 'oct4_encumberedheuristic/oct4_encumberedheuristic_3_base_undersell_allowed', 'oct4_encumberedheuristic/oct4_encumberedheuristic_4_base_grace_period', 'oct4_encumberedheuristic/oct4_encumberedheuristic_1_base', 'oct4_encumberedheuristic/oct4_encumberedheuristic_1_spite', 'oct4_encumberedheuristic/oct4_encumberedheuristic_0_risk_averse_no_activity', 'oct4_encumberedheuristic/oct4_encumberedheuristic_1_spite_tie_break', 'oct4_encumberedheuristic/oct4_encumberedheuristic_2_base_no_activity', 'oct4_encumberedheuristic/oct4_encumberedheuristic_2_base', 'oct4_encumberedheuristic/oct4_encumberedheuristic_4_base', 'oct4_encumbe

# For running the real dumb test case

In [ ]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

75


In [42]:
# num_iters = 999_000_000
# time_limit_seconds = int(3600 * 0.5)
# for config in pgrid:
#     CONFIG = 'cfr_init_2' # The YML network config folder for the main jobs
#     N_SEEDS = 1 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_999_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun24_cfr_gamut_7', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 8)
for config in pgrid:
    CONFIG = 'cfr_port_4' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 9000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='oct03_v1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 99596
Submitted batch job 99597
Submitted batch job 99598
Submitted batch job 99599
Submitted batch job 99600
Submitted batch job 99601
Submitted batch job 99602
Submitted batch job 99603
Submitted batch job 99604
Submitted batch job 99605
Submitted batch job 99606
Submitted batch job 99607
Submitted batch job 99608
Submitted batch job 99609
Submitted batch job 99610
Submitted batch job 99611
Submitted batch job 99612
Submitted batch job 99613
Submitted batch job 99614
Submitted batch job 99615
Submitted batch job 99616
Submitted batch job 99617
Submitted batch job 99618
Submitted batch job 99619
Submitted batch job 99620
Submitted batch job 99621
Submitted batch job 99622
Submitted batch job 99623
Submitted batch job 99624
Submitted batch job 99625
Submitted batch job 99626
Submitted batch job 99627
Submitted batch job 99628
Submitted batch job 99629
Submitted batch job 99630
Submitted batch job 99631
Submitted batch job 99632
Submitted batch job 99633
Submitted ba

In [ ]:
# PREFIX = 'sep1'
# sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
# print(sats_games)
# sats_games = \
#     [s for s in sats_games if ('hide' in s)]
# pgrid = ParameterGrid([
#     {
#         # Defaults
#         'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
#     },
# ])
# list(pgrid)

In [ ]:
### Test 3 players

PREFIX = 'sep1'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
# sats_games = [s for s in sats_games if 'hide' not in s and 'no_activity' not in s and 'undersell' not in s and 'speed' not in s and 'spite' not in s]
# sats_games = [s for s in sats_games if ('base' in s or 'grace' in s or 'activity' in s or 'speed' in s) and 'spite' not in s and 'risk' not in s]
sats_games = \
    [s for s in sats_games if ('hide' in s)]
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])

num_iters = 999_000_000
time_limit_seconds = int(3600 * 9)
for config in pgrid:
    CONFIG = 'cfr_port_2_no_ext' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 9000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='3players_sep1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
# num_iters = 10_000_000
# time_limit_seconds = 3600 * 4
# for config in pgrid:
#     CONFIG = 'cfr_external' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 250_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun23external', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
# num_iters = 30_000_000
# time_limit_seconds = 3600 * 4
# for config in pgrid:
#     CONFIG = 'cfr_outcome' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun23outcome_2', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
# for config in pgrid:
#     CONFIG = 'ppo_no_ent_jun14'
#     # CONFIG = 'ppo_may31_23' # The YML network config folder for the main jobs
#     # BR_CONFIG = 'ppo_may31_23' 
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br false --br_overrides "--total_timesteps 1_000_000"'        
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun14ent_3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)

In [44]:
!squeue --format="%.18i %.90j %.2t .%.10M%.6D %R"

             JOBID                                                                                       NAME ST .      TIME NODES NODELIST(REASON)
             99506 sep19_encumbered_sep19_encumbered_1_spite_undersell_allowed-cfr_port_4outcome_lin_avg_plus PD .      0:00     1 (Resources)
             99507 sep19_encumbered_sep19_encumbered_1_spite_undersell_allowed-cfr_port_4outcome_lin_avg_plus PD .      0:00     1 (Priority)
             99508 sep19_encumbered_sep19_encumbered_1_spite_undersell_allowed-cfr_port_4outcome_lin_avg_plus PD .      0:00     1 (Priority)
             99509 sep19_encumbered_sep19_encumbered_1_spite_undersell_allowed-cfr_port_4outcome_lin_avg_plus PD .      0:00     1 (Priority)
             99510 sep19_encumbered_sep19_encumbered_1_spite_undersell_allowed-cfr_port_4outcome_lin_avg_plus PD .      0:00     1 (Priority)
             99511 sep19_encumbered_sep19_encumbered_1_spite_undersell_allowed-cfr_port_4outcome_lin_avg_plus PD .      0:00     1 (Priority)

In [ ]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u newmanne --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [ ]:
check_queue()

In [45]:
!squeue | wc -l

331


In [ ]:
!squeue | grep "may30" | wc -l

In [ ]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "eval" | awk '{print $1}' | xargs scancel

In [ ]:
from compress_pickle import dumps, loads


In [ ]:
loads?

In [ ]:
from compress_pickle.compressers.registry import get_known_compressions

In [ ]:
get_known_compressions()

In [ ]:
x = EquilibriumSolverRunCheckpoint.objects.last().policy

In [ ]:
len(x)

In [ ]:
len(dumps(pickle.loads(x), compression='bz2'))

In [ ]:
len(pickle.dumps(pickle.loads(x)))

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().walltime

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().t

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().bestresponse_set.all()

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().policy)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().first().model)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().last().model)